In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv("transformed_league_data.csv")
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,gameid,side,pick1,pick2,pick3,pick4,pick5,ban1,ban2,ban3,ban4,ban5,result,num_counters,num_counters_banned,PGA,higher_PGA
0,2,2,2,FRA1TMNT1/450138,Blue,Lissandra,Rek'Sai,Xerath,Tristana,Janna,LeBlanc,Kassadin,Lee Sin,NaN,NaN,1,0,0,0,0
1,3,3,3,FRA1TMNT1/450138,Red,Rumble,Rengar,Syndra,Sivir,Annie,Gnar,Azir,Irelia,NaN,NaN,0,2,1,3,1
2,4,4,4,FRA1TMNT1/450168,Blue,Gnar,Lee Sin,Nidalee,Corki,Janna,Cassiopeia,Rek'Sai,Lissandra,NaN,NaN,1,0,0,0,0
3,5,5,5,FRA1TMNT1/450168,Red,Vladimir,Jarvan IV,Azir,Sivir,Annie,Rumble,Riven,Kassadin,NaN,NaN,0,1,0,1,1
4,6,6,6,FRA1TMNT1/450169,Blue,Lissandra,Rengar,Kassadin,Graves,Morgana,Lee Sin,Gnar,Jarvan IV,NaN,NaN,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121995,167131,2913,2913,LOLTMNT06_96870,Red,Darius,Sejuani,Akali,Varus,Rell,Ambessa,Skarner,Aurora,Braum,Alistar,0,0,2,2,1
121996,167136,2918,2918,LOLTMNT06_96989,Blue,Gnar,Amumu,Jayce,Ezreal,Nautilus,Maokai,K'Sante,Viktor,Sion,Jax,1,0,0,0,0
121997,167137,2919,2919,LOLTMNT06_96989,Red,Malphite,Zyra,Corki,Kai'Sa,Leona,Skarner,Ambessa,Vi,Rell,Rakan,0,1,0,1,1
121998,167138,2920,2920,LOLTMNT06_96993,Blue,Ambessa,Zyra,Jayce,Ezreal,Nautilus,Varus,Sejuani,Kalista,Yone,Smolder,0,0,1,1,0


In [3]:
bin_edges = list(range(11))
bin_counts = df["PGA"].value_counts().reindex(bin_edges, fill_value=0).reset_index()
bin_counts.columns = ["PGA", "Count"]

fig = px.bar(
    bin_counts,
    x="PGA",
    y="Count",
    title="Distribution of Pre Game Advantages",
    labels={"PGA": "PGA Value", "Count": "Count"},
    text_auto=True
)

fig.update_layout(
    xaxis=dict(type="category"),
    bargap=0.05  
)

fig.show()


In [4]:
import plotly.express as px

data = {
    "Category": ["Teans with PGA", "Teams with no PGA"],
    "Values": [
        df[df['PGA'] == 0].shape[0], 
        df[df['PGA'] == 1].shape[0], 
    ]
}

fig = px.pie(
    names=data["Category"],  
    values=data["Values"],   
    title="How common is PGA?"
)

# Show plot
fig.show()

"""
Note that there are more teams with PGA. Meaning, most teams either counter picked the enemy or banned one of their counters. 
This leads us to believe that players are considering the affect of counter matchups
"""

'\nNote that there are more teams with PGA. Meaning, most teams either counter picked the enemy or banned one of their counters. \nThis leads us to believe that players are considering the affect of counter matchups\n'

In [5]:
import plotly.express as px

data = {
    "Category": ["Wins with PGA = 0", "Wins with PGA > 0"],
    "Values": [
        df[(df['result'] == 1) & (df['PGA'] == 0)].shape[0], 
        df[(df['result'] == 1) & (df['PGA'] != 0)].shape[0]
    ]
}

fig = px.pie(
    names=data["Category"],  
    values=data["Values"],   
    title="Distribution of Winners - When PGA present / missing"
)

# Show plot
fig.show()


"""
Note that most winning teams have at least one PGA. As shown earlier, about 56.5% of teams have a PGA > 0. Thus, since the 
wins with PGA > 0 are greater than wins with PGA = 0, we can be led to assume that PGA has some sort of affect on wins. 
However, these results are for when PGA = 1. Lets see the distributions of wins for a higher PGA, lets say 5. 
"""

'\nNote that most winning teams have at least one PGA. As shown earlier, about 56.5% of teams have a PGA > 0. Thus, since the \nwins with PGA > 0 are greater than wins with PGA = 0, we can be led to assume that PGA has some sort of affect on wins. \nHowever, these results are for when PGA = 1. Lets see the distributions of wins for a higher PGA, lets say 5. \n'

In [6]:
import pandas as pd

df = df.sort_values(by=["gameid", "side"])
games = df.groupby("gameid")

win_higher_pga = 0
lose_higher_pga = 0

for gameid, group in games:
    if len(group) != 2:
        continue  # Skip if game data is incomplete

    team1, team2 = group.iloc[0], group.iloc[1]

    # Determine which team won
    if team1["result"] == 1:
        winner, loser = team1, team2
    else:
        winner, loser = team2, team1

    if winner["PGA"] > loser["PGA"]:
        win_higher_pga += 1
    elif loser["PGA"] > winner["PGA"]:
        lose_higher_pga += 1

labels = ["Won with Higher PGA", "Lost with Higher PGA"]
values = [win_higher_pga, lose_higher_pga]

fig = px.pie(
    names=labels, 
    values=values, 
    title="Dist of game result for teams with Higher PGA than opponent", 
)

fig.show()

"""
This visualization shows another 50/50 split. So, it looks like PGA doesnt have 
an affect on your wins/losses, if half of the teams are beating their opponent, while having less PGA
Then why did our distribution of winners have so many teams with PGA > 0? 

Well, when considering the number of champions per lane, usually about 20-30, and we're considering the top 5 counters, then
theres a 1/5 - 1/6 chance that a pick will counter an opponent. Additionally, it is very reasonable to say that a player 
will not want to play against their counter, leading to most players banning their counter. But lets see which variable affected PGA the most. 
"""

"\nThis visualization shows another 50/50 split. So, it looks like PGA doesnt have \nan affect on your wins/losses, if half of the teams are beating their opponent, while having less PGA\nThen why did our distribution of winners have so many teams with PGA > 0? \n\nWell, when considering the number of champions per lane, usually about 20-30, and we're considering the top 5 counters, then\ntheres a 1/5 - 1/6 chance that a pick will counter an opponent. Additionally, it is very reasonable to say that a player \nwill not want to play against their counter, leading to most players banning their counter. But lets see which variable affected PGA the most. \n"

In [7]:
import plotly.express as px

data = {
    "Category": ["PGA from counters", "PGA from bans"],
    "Values": [
        df['num_counters'].sum(), 
        df['num_counters_banned'].sum(), 
    ]
}

fig = px.pie(
    names=data["Category"],  
    values=data["Values"],   
    title="Distribution of PGA factor"
)

# Show plot
fig.show()


In [8]:
import plotly.express as px

data = {
    "Category": ["Wins on Red", "Wins on Blue"],
    "Values": [
        df[(df['result'] == 1) & (df['side'] == "Red")].shape[0], 
        df[(df['result'] == 1) & (df['side'] == "Blue")].shape[0], 
    ]
}

fig = px.pie(
    names=data["Category"],  
    values=data["Values"],   
    title="Distribution of Results for Side"
)

# Show plot
fig.show()

In [9]:
#bar chart, each bar is a PGA (0-10)
#each pga has two bars, # wins # losses 
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,gameid,side,pick1,pick2,pick3,pick4,pick5,ban1,ban2,ban3,ban4,ban5,result,num_counters,num_counters_banned,PGA,higher_PGA
2566,3632,0,0,1000-1175,Blue,Ekko,Hecarim,Malzahar,Ashe,Karma,LeBlanc,Trundle,Shen,NaN,NaN,0,1,0,1,1
2567,3633,1,1,1000-1175,Red,Gnar,Gragas,Viktor,Sivir,Braum,Janna,Maokai,Vladimir,NaN,NaN,1,0,0,0,0
89714,122608,0,0,10000-10000_game_1,Blue,Jax,Sejuani,Akali,Kai'Sa,Nautilus,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
89715,122609,1,1,10000-10000_game_1,Red,K'Sante,Vi,Galio,Zeri,Alistar,NaN,NaN,NaN,NaN,NaN,1,1,0,1,1
89716,122610,2,2,10000-10000_game_2,Blue,Jax,Maokai,Ryze,Xayah,Rakan,NaN,NaN,NaN,NaN,NaN,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16829,23455,11395,11395,WMC2TMNT1/300016,Red,Cho'Gath,Rek'Sai,Syndra,Kog'Maw,Rakan,Kalista,Jarvan IV,Cassiopeia,Gnar,Thresh,1,2,1,3,1
16830,23456,11396,11396,WMC2TMNT1/320030,Blue,Gnar,Jarvan IV,Malzahar,Xayah,Alistar,Taric,Azir,Varus,Karma,Janna,1,1,1,2,1
16831,23457,11397,11397,WMC2TMNT1/320030,Red,Cho'Gath,Sejuani,Ryze,Caitlyn,Braum,Ornn,Kalista,Tristana,Maokai,Shen,0,0,0,0,0
16832,23458,11398,11398,WMC2TMNT1/320038,Blue,Maokai,Jarvan IV,Syndra,Varus,Braum,Azir,Alistar,Tristana,Shen,Gnar,0,0,1,1,0


In [27]:
import plotly.graph_objects as go

pga_range = list(range(11))

win_PGAs = df[df['result'] == 1]['PGA'].value_counts()
loss_PGAs = df[df['result'] == 0]['PGA'].value_counts()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=pga_range,
    y=win_PGAs,
    name='Wins',
    marker_color='blue'
))

fig.add_trace(go.Bar(
    x=pga_range,
    y=loss_PGAs,
    name='Losses',
    marker_color='red'
))

fig.update_layout(
    title='Wins and Losses by PGA',
    xaxis=dict(
        title='PGA',
        tickmode='array',
        tickvals=pga_range  
    ),
    yaxis_title='Count',
    barmode='group'  
)

fig.show()
